In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

from keras.layers import Dense, Flatten, Dropout, Conv1D, MaxPooling1D, LSTM
from keras.models import Sequential
from keras.utils import to_categorical, plot_model

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix as pcm

import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

### Utils

In [ ]:
# Функции для разделение временного ряда на интервалы размера 64

def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += (size / 2)
        
def segment_signal(data, window_size = 64):
    segments = np.empty((0,window_size,6))
    labels = np.empty((0))
    for (start, end) in windows(data['timestamp'], window_size):
        acc_x = data['accX'][start:end]
        acc_y = data['accY'][start:end]
        acc_z = data['accZ'][start:end]
        gyr_x = data['gyrX'][start:end]
        gyr_y = data['gyrY'][start:end]
        gyr_z = data['gyrZ'][start:end]
        if(len(data['timestamp'][start:end]) == window_size):
            segments = np.vstack([segments,np.dstack([acc_x, acc_y, acc_z, gyr_x, gyr_y, gyr_z])])
            labels = np.append(labels,stats.mode(data['activityMode'][start:end])[0][0])
    return segments, labels

In [ ]:
def plot_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(14, 6))
    ax[0].plot(history.epoch, history.history['accuracy'], label='accuracy')
    ax[0].plot(history.epoch, history.history['val_accuracy'], label='val accuracy')
    ax[1].plot(history.epoch, history.history['loss'], label='loss')
    ax[1].plot(history.epoch, history.history['val_loss'], label='val loss')
    ax[0].legend()
    ax[1].legend()
    plt.show()

In [ ]:
def plot_confusion_matrix(model, test_x, test_y):
    pred = model.predict(test_x)
    pred = np.argmax(pred, axis = 1) 
    y_true = np.argmax(test_y, axis = 1)

    cm = confusion_matrix(y_true, pred)
    fig, ax = pcm(conf_mat=cm, figsize=(10, 5))
    plt.show()

### Load dataset

In [ ]:
df = pd.read_csv('SensorData/train_sens.csv')

In [ ]:
df.head()

In [ ]:
df['activityMode'].value_counts()

In [ ]:
plt.figure(figsize=(14, 3))
sns.kdeplot(data=df.iloc[:, 1:-2], fill=True, common_norm=False)

In [ ]:
segments, labels = segment_signal(df)
labels = np.asarray(pd.get_dummies(labels), dtype = np.int8)

In [ ]:
print(segments.shape)
print(labels.shape)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(segments, labels, test_size=0.20)

In [ ]:
n_timestamps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

### Ver. 1. Original data (CNN)

In [ ]:
model = Sequential()
model.add(Conv1D(128, 3, activation='relu', input_shape=(n_timestamps, n_features)))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, train_y, epochs=80, batch_size=10, validation_split=0.1)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
plot_history(history)

In [ ]:
model.save('models/cnn-ver-1.h5')

In [ ]:
plot_confusion_matrix(model, test_x, test_y)

### Ver. 2. Original data (LSTM)

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(n_timestamps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(train_x, train_y, epochs=80, batch_size=10, validation_split=0.1)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
plot_history(history)

In [ ]:
model.save('models/lstm-ver-1.h5')

In [ ]:
plot_confusion_matrix(model, test_x, test_y)